# Introduction

*Road safety is a very important topic around the world, and in the United Kingdom it's no different. Each year thousands of accidents are reported to the police and details such as location, date, weather, road conditions, and number of causalities are recorded. The consultant, Ian Jeffries, a Data Scientist working for the Department of Transport and Infrastructure, has been tasked with better understanding the root cause of high accident frequencies by "the client". There are three datasets to be analyzed: Accident information from 2017, Casualty information from 2017, and Vehicle information from 2017. These files are freely available for download on data.gov.uk. [1] The goal is to make roads safer for everyone by digging into common denominators and using visualization to make sense of the data. Seven specific questions were posed to the consultant, which will be addressed in the following brief.*

In [18]:
#import necessary packages
import pyspark, os, zipfile
import pandas as pd
import urllib.request

In [17]:
#perform get request to download files

#get accident data
urllib.request.urlretrieve('http://data.dft.gov.uk.s3.amazonaws.com/road-accidents-safety-data/dftRoadSafetyData_Accidents_2017.zip',
                          'C:/Users/ianke/Desktop/accidents.zip')

#get casualty data
urllib.request.urlretrieve('http://data.dft.gov.uk.s3.amazonaws.com/road-accidents-safety-data/dftRoadSafetyData_Casualties_2017.zip',
                          'C:/Users/ianke/Desktop/casualties.zip')

#get vehicle data
urllib.request.urlretrieve('http://data.dft.gov.uk.s3.amazonaws.com/road-accidents-safety-data/dftRoadSafetyData_Vehicles_2017.zip',
                          'C:/Users/ianke/Desktop/vehicle.zip')

#get dictionary info
urllib.request.urlretrieve('http://data.dft.gov.uk/road-accidents-safety-data/Road-Accident-Safety-Data-Guide.xls',
                          'C:/Users/ianke/Desktop/dictionary.xls')

('C:/Users/ianke/Desktop/dictionary.xls',
 <http.client.HTTPMessage at 0x1af0c0d0160>)

In [19]:
#unzip accident data
accidentZip = zipfile.ZipFile('C:/Users/ianke/Desktop/accidents.zip', 'r')
accidentZip.extractall('C:/Users/ianke/Desktop/')
accidentZip.close()

#unzip casualty data
casualtyZip = zipfile.ZipFile('C:/Users/ianke/Desktop/casualties.zip', 'r')
casualtyZip.extractall('C:/Users/ianke/Desktop/')
casualtyZip.close()

#unzip vehicle data
casualtyZip = zipfile.ZipFile('C:/Users/ianke/Desktop/vehicle.zip', 'r')
casualtyZip.extractall('C:/Users/ianke/Desktop/')
casualtyZip.close()

In [22]:
#delete zip folders
os.remove('C:/Users/ianke/Desktop/accidents.zip')
os.remove('C:/Users/ianke/Desktop/casualties.zip')
os.remove('C:/Users/ianke/Desktop/vehicle.zip')

## Query to Locate Top 3 Police Forces

Using the newly created dictionary, the data can be aggregated by most fatal accidents by police force. A value of '1' in the 'Accident_Severity' column indicates whether the accident was fatal. Using this information, a simple query returns the top three polices forces by number of fatal accidents. The results of this query will be saved in a newly created 'Reports' folder to create vizualisations.

## Visualizing Top 3 Police Forces by Most Fatal Accidents

Now that the query is saved in a csv file, the pandas [2] and matplotlib [4] packages can be used within python to create a simple bar chart of the results. (These packages will be used for all visualizations)

In [ ]:
#install nessesary packages on machine
#!sudo pip install matplotlib

#display visualization in jupyter output inline
%matplotlib inline

#import packages
import matplotlib.pyplot as plt
import pandas as pd

#read csv into dataframe
top_three = pd.read_csv('~/Desktop/Assignment_files/Reports/QuestionOne.csv')

#create bar chart
plt.bar(top_three.police_force, height=top_three.number_of_fatal, color='black')
plt.xlabel('Police Force')
plt.ylabel('Number of Fatal Accidents')
plt.title('Top 3 Most Fatal Accidents', fontdict = {'fontsize' : 16})
plt.style.use('ggplot')
plt.show()

## Conclusions

As seen above, the three Police Forces that have the most fatal accidents are the Metropolitan Police, Thames Valley Police Force, and Kent. The chart above shows that Metropolitan Police have substantially more than the next two Police Forces, with the total number of fatal accidents over double the next highest.

# Question Two

The second question wishes to find the top 5 local authorities that have the most serious accidents. There are two variables of interest: 'Accident_Severity' and 'Local_Authority_District'. To find all serious accidents, 'Accident_Severity' must be filtered to the value '2', which equals 'Serious'. The 'Local_Authority_District' variable can be used to find the top 5 local authority districts. A data dictionary will need to be created to identify which local authority relates to the coded values saved in the 'Accidents' database. 

## Data Dictionaries for Local Authority Variable

The following code creates a new dictionary table for the 'Local_Authority_District' variable:

In [ ]:
#create Local Authority District Data Dictionary
! impala-shell -q "USE roadSafety; \
                   CREATE TABLE local_Auth_Dist_Dict ( \
                        code int, \
                        Local_Authority_District string \
                   ) \
                   ROW FORMAT DELIMITED \
                   FIELDS TERMINATED BY ',';"

#insert values into dictionary
! impala-shell -q "USE roadSafety; \
                   INSERT INTO local_Auth_Dist_Dict \
                   VALUES \
                    (1, 'Westminster'), \
                    (2, 'Camden'), \
                    (3, 'Islington'), \
                    (4, 'Hackney'), \
                    (5, 'Tower Hamlets'), \
                    (6, 'Greenwich'), \
                    (7, 'Lewisham'), \
                    (8, 'Southwark'), \
                    (9, 'Lambeth'), \
                    (10, 'Wandsworth'), \
                    (11, 'Hammersmith and Fulham'), \
                    (12, 'Kensington and Chelsea'), \
                    (13, 'Waltham Forest'), \
                    (14, 'Redbridge'), \
                    (15, 'Havering'), \
                    (16, 'Barking and Dagenham'), \
                    (17, 'Newham'), \
                    (18, 'Bexley'), \
                    (19, 'Bromley'), \
                    (20, 'Croydon'), \
                    (21, 'Sutton'), \
                    (22, 'Merton'), \
                    (23, 'Kingston upon Thames'), \
                    (24, 'Richmond upon Thames'), \
                    (25, 'Hounslow'), \
                    (26, 'Hillingdon'), \
                    (27, 'Ealing'), \
                    (28, 'Brent'), \
                    (29, 'Harrow'), \
                    (30, 'Barnet'), \
                    (31, 'Haringey'), \
                    (32, 'Enfield'), \
                    (33, 'Hertsmere'), \
                    (38, 'Epsom and Ewell'), \
                    (40, 'Spelthorne'), \
                    (57, 'London Airport (Heathrow)'), \
                    (60, 'Allerdale'), \
                    (61, 'Barrow-in-Furness'), \
                    (62, 'Carlisle'), \
                    (63, 'Copeland'), \
                    (64, 'Eden'), \
                    (65, 'South Lakeland'), \
                    (70, 'Blackburn with Darwen'), \
                    (71, 'Blackpool'), \
                    (72, 'Burnley'), \
                    (73, 'Chorley'), \
                    (74, 'Fylde'), \
                    (75, 'Hyndburn'), \
                    (76, 'Lancaster'), \
                    (77, 'Pendle'), \
                    (79, 'Preston'), \
                    (80, 'Ribble Valley'), \
                    (82, 'Rossendale'), \
                    (83, 'South Ribble'), \
                    (84, 'West Lancashire'), \
                    (85, 'Wyre'), \
                    (90, 'Knowsley'), \
                    (91, 'Liverpool'), \
                    (92, 'St. Helens'), \
                    (93, 'Sefton'), \
                    (95, 'Wirral'), \
                    (100, 'Bolton'), \
                    (101, 'Bury'), \
                    (102, 'Manchester'), \
                    (104, 'Oldham'), \
                    (106, 'Rochdale'), \
                    (107, 'Salford'), \
                    (109, 'Stockport'), \
                    (110, 'Tameside'), \
                    (112, 'Trafford'), \
                    (114, 'Wigan'), \
                    (120, 'Chester'), \
                    (121, 'Congleton'), \
                    (122, 'Crewe and Nantwich'), \
                    (123, 'Ellesmere Port and Neston'), \
                    (124, 'Halton'), \
                    (126, 'Macclesfield'), \
                    (127, 'Vale Royal'), \
                    (128, 'Warrington'), \
                    (129, 'Cheshire East'), \
                    (130, 'Cheshire West and Chester'), \
                    (139, 'Northumberland'), \
                    (140, 'Alnwick'), \
                    (141, 'Berwick-upon-Tweed'), \
                    (142, 'Blyth Valley'), \
                    (143, 'Castle Morpeth'), \
                    (144, 'Tynedale'), \
                    (145, 'Wansbeck'), \
                    (146, 'Gateshead'), \
                    (147, 'Newcastle upon Tyne'), \
                    (148, 'North Tyneside'), \
                    (149, 'South Tyneside'), \
                    (150, 'Sunderland'), \
                    (160, 'Chester-le-Street'), \
                    (161, 'Darlington'), \
                    (162, 'Derwentside'), \
                    (163, 'Durham'), \
                    (164, 'Easington'), \
                    (165, 'Sedgefield'), \
                    (166, 'Teesdale'), \
                    (168, 'Wear Valley'), \
                    (169, 'County Durham'), \
                    (180, 'Craven'), \
                    (181, 'Hambleton'), \
                    (182, 'Harrogate'), \
                    (184, 'Richmondshire'), \
                    (185, 'Ryedale'), \
                    (186, 'Scarborough'), \
                    (187, 'Selby'), \
                    (189, 'York'), \
                    (200, 'Bradford'), \
                    (202, 'Calderdale'), \
                    (203, 'Kirklees'), \
                    (204, 'Leeds'), \
                    (206, 'Wakefield'), \
                    (210, 'Barnsley'), \
                    (211, 'Doncaster'), \
                    (213, 'Rotherham'), \
                    (215, 'Sheffield'), \
                    (228, 'Kingston upon Hull, City of'), \
                    (231, 'East Riding of Yorkshire'), \
                    (232, 'North Lincolnshire'), \
                    (233, 'North East Lincolnshire'), \
                    (240, 'Hartlepool'), \
                    (241, 'Redcar and Cleveland'), \
                    (243, 'Middlesbrough'), \
                    (245, 'Stockton-on-Tees'), \
                    (250, 'Cannock Chase'), \
                    (251, 'East Staffordshire'), \
                    (252, 'Lichfield'), \
                    (253, 'Newcastle-under-Lyme'), \
                    (254, 'South Staffordshire'), \
                    (255, 'Stafford'), \
                    (256, 'Staffordshire Moorlands'), \
                    (257, 'Stoke-on-Trent'), \
                    (258, 'Tamworth'), \
                    (270, 'Bromsgrove'), \
                    (273, 'Malvern Hills'), \
                    (274, 'Redditch'), \
                    (276, 'Worcester'), \
                    (277, 'Wychavon'), \
                    (278, 'Wyre Forest'), \
                    (279, 'Bridgnorth'), \
                    (280, 'North Shropshire'), \
                    (281, 'Oswestry'), \
                    (282, 'Shrewsbury and Atcham'), \
                    (283, 'South Shropshire'), \
                    (284, 'Telford and Wrekin'), \
                    (285, 'Herefordshire, County of '), \
                    (286, 'Shropshire'), \
                    (290, 'North Warwickshire'), \
                    (291, 'Nuneaton and Bedworth'), \
                    (292, 'Rugby '), \
                    (293, 'Stratford-upon-Avon'), \
                    (294, 'Warwick'), \
                    (300, 'Birmingham'), \
                    (302, 'Coventry'), \
                    (303, 'Dudley'), \
                    (305, 'Sandwell'), \
                    (306, 'Solihull'), \
                    (307, 'Walsall'), \
                    (309, 'Wolverhampton'), \
                    (320, 'Amber Valley'), \
                    (321, 'Bolsover'), \
                    (322, 'Chesterfield'), \
                    (323, 'Derby'), \
                    (324, 'Erewash'), \
                    (325, 'High Peak'), \
                    (327, 'North East Derbyshire'), \
                    (328, 'South Derbyshire'), \
                    (329, 'Derbyshire Dales'), \
                    (340, 'Ashfield'), \
                    (341, 'Bassetlaw'), \
                    (342, 'Broxtowe'), \
                    (343, 'Gedling'), \
                    (344, 'Mansfield'), \
                    (345, 'Newark and Sherwood'), \
                    (346, 'Nottingham'), \
                    (347, 'Rushcliffe'), \
                    (350, 'Boston'), \
                    (351, 'East Lindsey'), \
                    (352, 'Lincoln'), \
                    (353, 'North Kesteven'), \
                    (354, 'South Holland'), \
                    (355, 'South Kesteven'), \
                    (356, 'West Lindsey'), \
                    (360, 'Blaby'), \
                    (361, 'Hinckley and Bosworth'), \
                    (362, 'Charnwood'), \
                    (363, 'Harborough'), \
                    (364, 'Leicester'), \
                    (365, 'Melton'), \
                    (366, 'North West Leicestershire'), \
                    (367, 'Oadby and Wigston'), \
                    (368, 'Rutland'), \
                    (380, 'Corby'), \
                    (381, 'Daventry'), \
                    (382, 'East Northamptonshire'), \
                    (383, 'Kettering'), \
                    (384, 'Northampton'), \
                    (385, 'South Northamptonshire'), \
                    (386, 'Wellingborough'), \
                    (390, 'Cambridge'), \
                    (391, 'East Cambridgeshire'), \
                    (392, 'Fenland'), \
                    (393, 'Huntingdonshire'), \
                    (394, 'Peterborough'), \
                    (395, 'South Cambridgeshire'), \
                    (400, 'Breckland'), \
                    (401, 'Broadland'), \
                    (402, 'Great Yarmouth'), \
                    (404, 'Norwich'), \
                    (405, 'North Norfolk'), \
                    (406, 'South Norfolk'), \
                    (407, 'Kings Lynn and West Norfolk'), \
                    (410, 'Babergh'), \
                    (411, 'Forest Heath'), \
                    (412, 'Ipswich'), \
                    (413, 'Mid Suffolk'), \
                    (414, 'St. Edmundsbury'), \
                    (415, 'Suffolk Coastal'), \
                    (416, 'Waveney'), \
                    (420, 'Bedford'), \
                    (421, 'Luton'), \
                    (422, 'Mid Bedfordshire'), \
                    (423, 'South Bedfordshire'), \
                    (424, 'Central Bedfordshire'), \
                    (430, 'Broxbourne'), \
                    (431, 'Dacorum'), \
                    (432, 'East Hertfordshire'), \
                    (433, 'North Hertfordshire'), \
                    (434, 'St. Albans'), \
                    (435, 'Stevenage'), \
                    (436, 'Three Rivers'), \
                    (437, 'Watford'), \
                    (438, 'Welwyn Hatfield'), \
                    (450, 'Basildon'), \
                    (451, 'Braintree'), \
                    (452, 'Brentwood'), \
                    (453, 'Castle Point'), \
                    (454, 'Chelmsford'), \
                    (455, 'Colchester'), \
                    (456, 'Epping Forest'), \
                    (457, 'Harlow'), \
                    (458, 'Maldon'), \
                    (459, 'Rochford'), \
                    (460, 'Southend-on-Sea'), \
                    (461, 'Tendring'), \
                    (462, 'Thurrock'), \
                    (463, 'Uttlesford'), \
                    (470, 'Bracknell Forest'), \
                    (471, 'West Berkshire'), \
                    (472, 'Reading'), \
                    (473, 'Slough'), \
                    (474, 'Windsor and Maidenhead'), \
                    (475, 'Wokingham'), \
                    (476, 'Aylesbury Vale'), \
                    (477, 'South Bucks'), \
                    (478, 'Chiltern'), \
                    (479, 'Milton Keynes'), \
                    (480, 'Wycombe'), \
                    (481, 'Cherwell'), \
                    (482, 'Oxford'), \
                    (483, 'Vale of White Horse'), \
                    (484, 'South Oxfordshire'), \
                    (485, 'West Oxfordshire'), \
                    (490, 'Basingstoke and Deane'), \
                    (491, 'Eastleigh'), \
                    (492, 'Fareham'), \
                    (493, 'Gosport'), \
                    (494, 'Hart'), \
                    (495, 'Havant'), \
                    (496, 'New Forest'), \
                    (497, 'East Hampshire'), \
                    (498, 'Portsmouth'), \
                    (499, 'Rushmoor'), \
                    (500, 'Southampton '), \
                    (501, 'Test Valley'), \
                    (502, 'Winchester'), \
                    (505, 'Isle of Wight'), \
                    (510, 'Elmbridge'), \
                    (511, 'Guildford'), \
                    (512, 'Mole Valley'), \
                    (513, 'Reigate and Banstead'), \
                    (514, 'Runnymede'), \
                    (515, 'Surrey Heath'), \
                    (516, 'Tandridge'), \
                    (517, 'Waverley'), \
                    (518, 'Woking'), \
                    (530, 'Ashford'), \
                    (531, 'Canterbury'), \
                    (532, 'Dartford'), \
                    (533, 'Dover'), \
                    (535, 'Gravesham'), \
                    (536, 'Maidstone'), \
                    (538, 'Sevenoaks'), \
                    (539, 'Shepway'), \
                    (540, 'Swale'), \
                    (541, 'Thanet'), \
                    (542, 'Tonbridge and Malling'), \
                    (543, 'Tunbridge Wells'), \
                    (544, 'Medway'), \
                    (551, 'Eastbourne'), \
                    (552, 'Hastings'), \
                    (554, 'Lewes'), \
                    (555, 'Rother'), \
                    (556, 'Wealden'), \
                    (557, 'Adur'), \
                    (558, 'Arun'), \
                    (559, 'Chichester'), \
                    (560, 'Crawley'), \
                    (562, 'Horsham'), \
                    (563, 'Mid Sussex'), \
                    (564, 'Worthing'), \
                    (565, 'Brighton and Hove'), \
                    (570, 'City of London'), \
                    (580, 'East Devon'), \
                    (581, 'Exeter'), \
                    (582, 'North Devon'), \
                    (583, 'Plymouth'), \
                    (584, 'South Hams'), \
                    (585, 'Teignbridge'), \
                    (586, 'Mid Devon'), \
                    (587, 'Torbay'), \
                    (588, 'Torridge'), \
                    (589, 'West Devon'), \
                    (590, 'Caradon'), \
                    (591, 'Carrick'), \
                    (592, 'Kerrier'), \
                    (593, 'North Cornwall'), \
                    (594, 'Penwith'), \
                    (595, 'Restormel'), \
                    (596, 'Cornwall'), \
                    (601, 'Bristol, City of'), \
                    (605, 'North Somerset'), \
                    (606, 'Mendip'), \
                    (607, 'Sedgemoor'), \
                    (608, 'Taunton Deane'), \
                    (609, 'West Somerset'), \
                    (610, 'South Somerset'), \
                    (611, 'Bath and North East Somerset'), \
                    (612, 'South Gloucestershire'), \
                    (620, 'Cheltenham'), \
                    (621, 'Cotswold'), \
                    (622, 'Forest of Dean'), \
                    (623, 'Gloucester'), \
                    (624, 'Stroud'), \
                    (625, 'Tewkesbury'), \
                    (630, 'Kennet'), \
                    (631, 'North Wiltshire'), \
                    (632, 'Salisbury'), \
                    (633, 'Swindon'), \
                    (634, 'West Wiltshire'), \
                    (635, 'Wiltshire'), \
                    (640, 'Bournemouth'), \
                    (641, 'Christchurch'), \
                    (642, 'North Dorset'), \
                    (643, 'Poole'), \
                    (644, 'Purbeck'), \
                    (645, 'West Dorset'), \
                    (646, 'Weymouth and Portland'), \
                    (647, 'East Dorset'), \
                    (720, 'Isle of Anglesey'), \
                    (721, 'Conwy'), \
                    (722, 'Gwynedd'), \
                    (723, 'Denbighshire'), \
                    (724, 'Flintshire'), \
                    (725, 'Wrexham'), \
                    (730, 'Blaenau Gwent'), \
                    (731, 'Caerphilly'), \
                    (732, 'Monmouthshire'), \
                    (733, 'Newport'), \
                    (734, 'Torfaen'), \
                    (740, 'Bridgend'), \
                    (741, 'Cardiff'), \
                    (742, 'Merthyr Tydfil'), \
                    (743, 'Neath Port Talbot'), \
                    (744, 'Rhondda, Cynon, Taff'), \
                    (745, 'Swansea'), \
                    (746, 'The Vale of Glamorgan'), \
                    (750, 'Ceredigion'), \
                    (751, 'Carmarthenshire'), \
                    (752, 'Pembrokeshire'), \
                    (753, 'Powys'), \
                    (910, 'Aberdeen City'), \
                    (911, 'Aberdeenshire'), \
                    (912, 'Angus'), \
                    (913, 'Argyll and Bute'), \
                    (914, 'Scottish Borders'), \
                    (915, 'Clackmannanshire'), \
                    (916, 'West Dunbartonshire'), \
                    (917, 'Dumfries and Galloway'), \
                    (918, 'Dundee City'), \
                    (919, 'East Ayrshire'), \
                    (920, 'East Dunbartonshire'), \
                    (921, 'East Lothian'), \
                    (922, 'East Renfrewshire'), \
                    (923, 'Edinburgh, City of'), \
                    (924, 'Falkirk'), \
                    (925, 'Fife'), \
                    (926, 'Glasgow City'), \
                    (927, 'Highland'), \
                    (928, 'Inverclyde'), \
                    (929, 'Midlothian'), \
                    (930, 'Moray'), \
                    (931, 'North Ayrshire'), \
                    (932, 'North Lanarkshire'), \
                    (933, 'Orkney Islands'), \
                    (934, 'Perth and Kinross'), \
                    (935, 'Renfrewshire'), \
                    (936, 'Shetland Islands'), \
                    (937, 'South Ayrshire'), \
                    (938, 'South Lanarkshire'), \
                    (939, 'Stirling'), \
                    (940, 'West Lothian'), \
                    (941, 'Western Isles');"

## Query to Locate Top 5 Local Authorities

Using the newly created dictionary, the data can be aggregated by local authority. A value of '2' in the 'Accident_Severity' column indicates whether the accident was serious. A query returns the top five local authorities by number of serious accidents. The results of this query will be saved in the 'Reports' folder.

In [ ]:
#query to find top 5 local authorities by serious accidents
! impala-shell -q "SELECT d.local_authority_district, COUNT(*) AS Number_of_Serious \
                   FROM roadSafety.accidents AS a \
                    LEFT JOIN roadSafety.local_auth_dist_dict AS d \
                    ON a.local_authority_district = d.code \
                   WHERE a.Accident_Severity = 2 \
                   GROUP BY d.local_authority_district \
                   ORDER BY Number_of_Serious DESC \
                   LIMIT 5;" \
                
#export to CSV
! impala-shell -B \
               -o ~/Desktop/Assignment_files/Reports/QuestionTwo.csv \
               --print_header \
               --output_delimiter=',' \
               -q "SELECT d.local_authority_district AS Local_Authority, COUNT(*) AS Number_of_Serious \
                   FROM roadSafety.accidents AS a \
                    LEFT JOIN roadSafety.local_auth_dist_dict AS d \
                    ON a.local_authority_district = d.code \
                   WHERE a.Accident_Severity = 2 \
                   GROUP BY d.local_authority_district \
                   ORDER BY Number_of_Serious DESC \
                   LIMIT 5;" \

## Visualizing Top 5 Local Authorities by Most Serious Accidents

Now that the query is saved in a csv file, the 'pandas' and 'Matplotlib' packages can be used within python to create a simple bar chart of the results:

In [ ]:
#display visualization in jupyter output inline
%matplotlib inline

#import packages
import matplotlib.pyplot as plt
import pandas as pd

#read csv into dataframe
top_five = pd.read_csv('~/Desktop/Assignment_files/Reports/QuestionTwo.csv')

#create bar chart
plt.bar(top_five.local_authority, height=top_five.number_of_serious, color='black')
plt.xlabel('Local Authority District')
plt.ylabel('Number of Serious Accidents')
plt.title('Top 5 Most Serious Accidents', fontdict = {'fontsize' : 16})
plt.style.use('ggplot')
plt.show()

## Conclusions

The above chart shows the top 5 Local Authority Districts with the most serious accidents. Birmingham leads the way with 375, followed closely by Leeds with 291, Sheffield with 279, Westminster with 259 and Cornwall with 224. 

# Question Three

The third question relates to the vehicle types that have been involved in the most serious and fatal accidents. There are two variables of interest: 'Accident_Severity' and 'Vehicle_Type'. To find all serious and fatal accidents, 'Accident_Severity' must be filtered to the values '1' and '2'. The 'Vehicle_Type' variable is stored as coded values, and must be defined with a dictionary. The 'accidents' and 'vehicles' tables will need to be joined to bring in both variables. 

## Creating Vehicle Type Data Dictionary

In [ ]:
#create Vehicle Type Data dictionary
! impala-shell -q "USE roadSafety; \
                   CREATE TABLE vehicle_Type_Dict ( \
                        code int, \
                        Vehicle_type string \
                   ) \
                   ROW FORMAT DELIMITED \
                   FIELDS TERMINATED BY ',';"

#insert values into dictionary
! impala-shell -q "USE roadSafety; \
                   INSERT INTO vehicle_Type_Dict \
                   VALUES \
                    (1, 'Pedal cycle'), \
                    (2, 'Motorcycle 50cc and under'), \
                    (3, 'Motorcycle 125cc and under'), \
                    (4, 'Motorcycle over 125cc and up to 500cc'), \
                    (5, 'Motorcycle over 500cc'), \
                    (8, 'Taxi/Private hire car'), \
                    (9, 'Car'), \
                    (10, 'Minibus (8 - 16 passenger seats)'), \
                    (11, 'Bus or coach (17 or more pass seats)'), \
                    (16, 'Ridden horse'), \
                    (17, 'Agricultural vehicle'), \
                    (18, 'Tram'), \
                    (19, 'Van / Goods 3.5 tonnes mgw or under'), \
                    (20, 'Goods over 3.5t. and under 7.5t'), \
                    (21, 'Goods 7.5 tonnes mgw and over'), \
                    (22, 'Mobility scooter'), \
                    (23, 'Electric motorcycle'), \
                    (90, 'Other vehicle'), \
                    (97, 'Motorcycle - unknown cc'), \
                    (98, 'Goods vehicle - unknown weight'), \
                    (-1, 'Data missing or out of range');"

## Query to Find Top Vehicle Types in Serious or Fatal Accidents

Now that a dictionary is created, both the 'accidents' table and the 'vehicle_type_dict' tables can be joined to the 'vehicle' table. 'Accident_Severity' can be filtered down to a 1 or 2, and all missing vehicle types can be removed. The following query shows the number of serious or fatal accidents broken down by vehicle type.

In [ ]:
#query to find top vehicles involved in serious or fatal accidents
! impala-shell -q "SELECT d.vehicle_type, COUNT(*) AS Serious_and_Fatal_Accidents \
                   FROM roadSafety.vehicles AS v \
                     INNER JOIN roadSafety.accidents AS a \
                     ON a.accident_index = v.accident_index \
                     LEFT JOIN roadSafety.vehicle_type_dict AS d \
                     ON v.vehicle_type = d.code \
                   WHERE (a.Accident_Severity = 1 OR a.Accident_Severity = 2) \
                          AND d.vehicle_type <> 'Data missing or out of range' \
                   GROUP BY d.vehicle_type \
                   ORDER BY Serious_and_Fatal_Accidents DESC;"
                
#export to CSV
! impala-shell -B \
               -o ~/Desktop/Assignment_files/Reports/QuestionThree.csv \
               --print_header \
               --output_delimiter=',' \
               -q "SELECT d.vehicle_type, COUNT(*) AS Serious_and_Fatal_Accidents \
                   FROM roadSafety.vehicles AS v \
                     INNER JOIN roadSafety.accidents AS a \
                     ON a.accident_index = v.accident_index \
                     LEFT JOIN roadSafety.vehicle_type_dict AS d \
                     ON v.vehicle_type = d.code \
                   WHERE (a.Accident_Severity = 1 OR a.Accident_Severity = 2) \
                          AND d.vehicle_type <> 'Data missing or out of range' \
                   GROUP BY d.vehicle_type \
                   ORDER BY Serious_and_Fatal_Accidents DESC;"

## Visualizing Vehicle Types Involved in Serious or Fatal Accidents

Since the total amount of accidents for many different vehicle types are being compared, a simple bar chart will display the information cleanly. To highlight the vehicle type with the most serious or fatal accidents, a red color bar was added. 

In [ ]:
#display visualization in jupyter output inline
%matplotlib inline

#import packages
import matplotlib.pyplot as plt
import pandas as pd

#read csv into dataframe
vehicles = pd.read_csv('~/Desktop/Assignment_files/Reports/QuestionThree.csv')

#reverse order so max is on top
vehicles = vehicles[::-1]

#add colors to highlight maximum column 
colors = ['black'] * 20
colors[19] = 'red'

#create bar chart (horizontal to fit the long label information)
plt.barh(vehicles.vehicle_type, width=vehicles.serious_and_fatal_accidents, color=colors)
plt.xlabel('Number of Serious or Fatal Accidents')
plt.ylabel('Vehicle Type')
plt.title('Number of Accidents By Vehicle Type', fontdict = {'fontsize' : 16})
plt.style.use('ggplot')
plt.show()

## Conclusions

As seen above, the vehicle type of 'Car' easily has the greatest number of fatal or serious accidents in 2017, with 21,525 more accidents than the next highest vehicle type. Intuitively, this makes sense as there are many more people driving cars than pedal cycles or motorcycles.

# Question Four

The fourth task is to find the age band of casualty that have the most fatal accidents. There are two variables of interest: 'Accident_Severity' and 'Age_Band_of_Casualty'. To find all fatal accidents, 'Accident_Severity' must be filtered to a '1'. The 'Age_Band_of_Casualty' variable is stored as coded values, and must be defined with a dictionary. The 'accidents' and 'casualties' tables will need to be joined to bring in both variables.  

## Creating Age Band of Casualty Data Dictionary

In [ ]:
#create age band Data dictionary
! impala-shell -q "USE roadSafety; \
                   CREATE TABLE age_band_dict ( \
                        code int, \
                        Age_Band_of_Casualty string \
                   ) \
                   ROW FORMAT DELIMITED \
                   FIELDS TERMINATED BY ',';"

#insert values into dictionary
! impala-shell -q "USE roadSafety; \
                   INSERT INTO age_band_dict \
                   VALUES \
                    (1, '0 - 5'), \
                    (2, '6 - 10'), \
                    (3, '11 - 15'), \
                    (4, '16 - 20'), \
                    (5, '21 - 25'), \
                    (6, '26 - 35'), \
                    (7, '36 - 45'), \
                    (8, '46 - 55'), \
                    (9, '56 - 65'), \
                    (10, '66 - 75'), \
                    (11, 'Over 75'), \
                    (-1, 'Data missing or out of range');"

## Query to Find Fatal Accidents by Age Band of Casualty

By joining the dictionary table and accidents table, the Age Band of Casualty variable can be grouped by total number of Fatal Accidents. Bringing in the Age Band code allows for sorting by code number, which will neatly order the age band groups for visualization. A filter can be added to only return accidents that were fatal and to remove data with a missing 'age band of causalty' label.

In [ ]:
#query to find fatal accidents by age band of casualty
! impala-shell -q "SELECT c.age_band_of_casualty AS age_code, d.age_band_of_casualty, COUNT(*) AS Fatal_Accidents \
                   FROM roadSafety.casualties AS c \
                     INNER JOIN roadSafety.accidents AS a \
                     ON a.accident_index = c.accident_index \
                     LEFT JOIN roadSafety.age_band_dict AS d \
                     ON c.age_band_of_casualty = d.code \
                   WHERE a.Accident_Severity = 1 \
                         AND d.age_band_of_casualty <> 'Data missing or out of range' \
                   GROUP BY c.age_band_of_casualty, d.age_band_of_casualty \
                   ORDER BY c.age_band_of_casualty;"
                
#export to CSV
! impala-shell -B \
               -o ~/Desktop/Assignment_files/Reports/QuestionFour.csv \
               --print_header \
               --output_delimiter=',' \
               -q "SELECT c.age_band_of_casualty AS age_code, d.age_band_of_casualty, COUNT(*) AS Fatal_Accidents \
                   FROM roadSafety.casualties AS c \
                     INNER JOIN roadSafety.accidents AS a \
                     ON a.accident_index = c.accident_index \
                     LEFT JOIN roadSafety.age_band_dict AS d \
                     ON c.age_band_of_casualty = d.code \
                   WHERE a.Accident_Severity = 1 \
                         AND d.age_band_of_casualty <> 'Data missing or out of range' \
                   GROUP BY c.age_band_of_casualty, d.age_band_of_casualty \
                   ORDER BY c.age_band_of_casualty;"

## Visualizing Fatal Accidents by Age Band of Casualty

Rather than return a single value for age band of casualty with the most fatal accidents, it was decided that a vizualization showing the trends by age group would be more beneficial for the client. To solve this, a shaded line chart was created.

In [ ]:
#display visualization in jupyter output inline
%matplotlib inline

#import packages
import matplotlib.pyplot as plt
import pandas as pd

#read csv into dataframe
age_band = pd.read_csv('~/Desktop/Assignment_files/Reports/QuestionFour.csv')

#increase width of chart 
plt.rcParams['figure.figsize'] = [8, 5]

#create line chart
plt.plot(age_band.age_band_of_casualty, age_band.fatal_accidents, color='black')
plt.xlabel('Age Band of Casualty', fontdict = {'fontsize' : 13})
plt.ylabel('Number of Fatal Accidents', fontdict = {'fontsize' : 13})
plt.title('Number of Fatal Accidents By Age Band', fontdict = {'fontsize' : 16})
plt.style.use('ggplot')
#rotate x labels so they are readable
plt.xticks(rotation=310)
#fill plot to simulate bar plot style
plt.fill_between(age_band.age_code - 1, age_band.fatal_accidents, color="black", alpha=0.4)
#mark the value of the highest point
plt.annotate('504', xy=(5, 504), xytext=(6, 480), arrowprops={'color':'black', 'width':1.7, 'headwidth':7})
plt.show()

## Conclusions

As seen in the chart above, there is a steep increase in total number of fatal accidents for the 'Age Band of Casualty' from 26-35, and that this group is involved in the highest number of fatal accidents. The lowest group is from 0-15, which would make sense as this age group is not yet driving. The only way they would be involved in the accident is if they are riding in the car, and the majority of drivers are commuting to work without children. The data also shows that the total number of accidents begin to trend upward as the age band increases over 75. 

# Question Five

The fifth task is to group and display the total number of slight accidents by day of week in the West Yorkshire Police Force Area. The variables of interest are 'Accident Severity', 'Day of Week', and 'Police Force'. To find all slight accidents, 'Accident Severity' must be filtered to a '3'. The Police Force dictionary can be used to filter down to the West Yorkshire Police Force, and 'Day of Week' can be calculated from the 'Accident_Date' field, since an earlier step converted that field into a Hive date format. All variables of interest are located in the 'accidents' table. 

## Query to Find Slight Accidents by Day of Week in West Yorkshire PFA

By joining in the dictionary table, the data can be manipulated in a readable way. Bringing in the day of week number allows for rational sorting, and a filter can be created to limit the data to slight accidents in the West Yorkshire police force area. 

In [ ]:
#query to find slight accidents by day of week in West Yorkshire PFA
! impala-shell -q "SELECT DAYOFWEEK(a.accident_date) AS Day_Number, \
                          DAYNAME(a.accident_date) AS Day_of_Week, \
                          COUNT(*) AS Slight_Accidents \
                   FROM roadSafety.accidents AS a \
                     LEFT JOIN roadSafety.police_force_dict AS d1 \
                     ON a.Police_Force = d1.code \
                   WHERE a.accident_severity = 3 AND d1.police_force = 'West Yorkshire' \
                   GROUP BY Day_Number, Day_of_Week \
                   ORDER BY Day_Number;"
                
#export to CSV
! impala-shell -B \
               -o ~/Desktop/Assignment_files/Reports/QuestionFive.csv \
               --print_header \
               --output_delimiter=',' \
               -q "SELECT DAYOFWEEK(a.accident_date) AS Day_Number, \
                          DAYNAME(a.accident_date) AS Day_of_Week, \
                          COUNT(*) AS Slight_Accidents \
                   FROM roadSafety.accidents AS a \
                     LEFT JOIN roadSafety.police_force_dict AS d1 \
                     ON a.Police_Force = d1.code \
                   WHERE a.accident_severity = 3 AND d1.police_force = 'West Yorkshire' \
                   GROUP BY Day_Number, Day_of_Week \
                   ORDER BY Day_Number;"

## Visualizing Slight Accidents by Day of Week in West Yorkshire PFA

A simple bar chart compares the slight accidents by day of week.

In [ ]:
#display visualization in jupyter output inline
%matplotlib inline

#import packages
import matplotlib.pyplot as plt
import pandas as pd

#read csv into dataframe
day_of_week = pd.read_csv('~/Desktop/Assignment_files/Reports/QuestionFive.csv')

#create bar chart
plt.bar(day_of_week.day_of_week, height=day_of_week.slight_accidents, color='black')
plt.xlabel('Day of Week')
plt.ylabel('Number of Slight Accidents')
plt.title('Accidents in West Yorkshire PFA', fontdict = {'fontsize' : 16})
plt.style.use('ggplot')
#rotate x labels so they are readable
plt.xticks(rotation=340)
plt.show()

## Conclusions

As seen above, the day of week with the highest number of slight accidents in the West Yorkshire Police Force Area is Friday, by a fairly significant margin. This is probably due to most people leaving to travel on Friday for the weekend. 

# Question Six

The sixth task is to find the peak hour that have the most fatal accidents in dual carriageway. All variables of interest are housed within the 'accidents' table and include 'Accident_Severity', 'Time' and 'Road_Type'. A filter of '1' must be applied to 'Accident_Severity' to retrieve all fatal accidents, and a filter of '3' will be applied to 'Road_Type' to return all accidents that occured on a dual carriageway. 

## Query to find Peak Hour with Most Fatal Accidents in Dual Carriageway

The 'time' variable reports time down to the minute and grouping by this variable wouldn't convey the trends the client is looking for, as accidents rarely occur within the same minute. This would lead to a graph that is mostly flat and would be very difficult to distinguish which hours have a higher number of accidents. It would be better to round the time variable to the nearest half hour and display the data as accidents by every half hour. The data is stored as text, so a nested IF statement is required to accurately round and format each time interval. The following query checks which time range the minutes fall into, and rounds to the nearest half hour accordingly. Casting is required to compare the minute text to integer values (minutes taken from the right side of the 'time' variable), and then convert back to string format for concatenation. Displaying the data in this way will allow for time series analysis in the visualization step. 

In [ ]:
#query to find peak hour with most fatal accidents in dual carriageway (round to nearest half-hour)
! impala-shell -q \
"SELECT \
     if(CAST(STRRIGHT(time, 2) AS int) BETWEEN 0 AND 15, CONCAT(STRLEFT(time, 2), ':', '00'), \
        if(CAST(STRRIGHT(time, 2) AS int) BETWEEN 16 AND 44, CONCAT(STRLEFT(time, 2), ':', '30'), \
           if(CAST(STRRIGHT(time, 2) AS int) BETWEEN 45 AND 60, if(CAST(STRLEFT(time, 2) AS int) + 1 < 10, \
               CONCAT('0', CAST((CAST(STRLEFT(time, 2) AS int) + 1) AS string), ':', '00'), \
               CONCAT(CAST((CAST(STRLEFT(time, 2) AS int) + 1) AS string), ':', '00')), 'Error'))) AS Accident_Time, \
     COUNT(*) AS Fatal_Accidents \
FROM roadSafety.accidents \
WHERE accident_severity = 1 AND road_type = 3 \
GROUP BY Accident_Time \
ORDER BY Accident_Time;"
                
#export to CSV
! impala-shell -B \
               -o ~/Desktop/Assignment_files/Reports/QuestionSix.csv \
               --print_header \
               --output_delimiter=',' \
-q "SELECT \
     if(CAST(STRRIGHT(time, 2) AS int) BETWEEN 0 AND 15, CONCAT(STRLEFT(time, 2), ':', '00'), \
        if(CAST(STRRIGHT(time, 2) AS int) BETWEEN 16 AND 44, CONCAT(STRLEFT(time, 2), ':', '30'), \
           if(CAST(STRRIGHT(time, 2) AS int) BETWEEN 45 AND 60, if(CAST(STRLEFT(time, 2) AS int) + 1 < 10, \
               CONCAT('0', CAST((CAST(STRLEFT(time, 2) AS int) + 1) AS string), ':', '00'), \
               CONCAT(CAST((CAST(STRLEFT(time, 2) AS int) + 1) AS string), ':', '00')), 'Error'))) AS Accident_Time, \
     COUNT(*) AS Fatal_Accidents \
FROM roadSafety.accidents \
WHERE accident_severity = 1 AND road_type = 3 \
GROUP BY Accident_Time \
ORDER BY Accident_Time;"                

## Visualizing Peak Hour with Most Fatal Accidents

The data is now in a format that allows for time series analysis. A line chart will accurately portray the trends in a 24 hour period, and should highlight the hour in which the most fatal accidents occur. 

In [ ]:
#display visualization in jupyter output inline
%matplotlib inline

#import packages
import matplotlib.pyplot as plt
import pandas as pd

#read csv into dataframe
peak_hour = pd.read_csv('~/Desktop/Assignment_files/Reports/QuestionSix.csv')

#increase width of chart to account for labels
plt.rcParams['figure.figsize'] = [13, 8]

#create line chart
plt.plot(peak_hour.accident_time, peak_hour.fatal_accidents, color='black')
plt.xlabel('Accident Time (Rounded to Half Hour)', fontdict = {'fontsize' : 16})
plt.ylabel('Number of Fatal Accidents', fontdict = {'fontsize' : 16})
plt.title('Fatal Accidents in Dual Carriageway', fontdict = {'fontsize' : 24})
plt.style.use('ggplot')

#rotate x labels so they are readable
plt.xticks(rotation=300)

#add arrow annotation
plt.annotate('17', xy=('20:00', 17), xytext=('21:00', 16), arrowprops={'color':'black', 'width':1.8, 'headwidth':8})

plt.show()

## Conclusions

As seen above, the most fatal accidents in dual carriageway occur at 20:00 with a total of 17 accidents. The next highest was 18:00 with a total of 13 accidents. It is interesting that the most accidents occur in the evening, when visibility might be bad, or people are tired driving home from work.

# Question Seven

The final question posed by the client was to find the area that has most fatal motorcycle accidents. The variables of interest from the 'accidents' table are 'Accident_Severity' and 'Urban_or_Rural_Area'. The variable of interest from the 'vehicles' table is 'Vehicle_Type'. By joining these tables together, it can be determined whether urban or rural areas have more fatal motorcycle accidents. 

## Query to Find Fatal Motorcycle Accidents by Area

The following query filters down to all fatal accidents where the vehicle involved was a motorcycle. A '1' in the 'Accident_Severity' variable selects all fatal accidents, and the LIKE operator filters down to any vehicle type in the dictionary table that begins with 'Motorcycle'. (There are 5 vehicle types that contain the word 'Motorcycle') Since there are only two codes in the 'Urban_or_Rural_Area' variable, a simple IF statement will decode them into a readable format for the viewer. 

In [ ]:
#query to find fatal motorcycle accidents by area
! impala-shell -q "SELECT IF(a.urban_or_rural_area = 1, 'Urban', \
                          IF(a.urban_or_rural_area = 2, 'Rural', 'Unallocated')) AS Area, \
                   COUNT(*) AS Fatal_Motorcycle_Accidents \
                   FROM roadsafety.accidents AS a \
                     LEFT JOIN roadsafety.vehicles AS v \
                     ON a.accident_index = v.accident_index \
                     LEFT JOIN roadsafety.vehicle_type_dict AS d \
                     ON v.vehicle_type = d.code \
                   WHERE a.accident_severity = 1 AND d.vehicle_type LIKE 'Motorcycle%' \
                   GROUP BY Area;"
                
#export to CSV
! impala-shell -B \
               -o ~/Desktop/Assignment_files/Reports/QuestionSeven.csv \
               --print_header \
               --output_delimiter=',' \
               -q "SELECT IF(a.urban_or_rural_area = 1, 'Urban', \
                          IF(a.urban_or_rural_area = 2, 'Rural', 'Unallocated')) AS Area, \
                   COUNT(*) AS Fatal_Motorcycle_Accidents \
                   FROM roadsafety.accidents AS a \
                     LEFT JOIN roadsafety.vehicles AS v \
                     ON a.accident_index = v.accident_index \
                     LEFT JOIN roadsafety.vehicle_type_dict AS d \
                     ON v.vehicle_type = d.code \
                   WHERE a.accident_severity = 1 AND d.vehicle_type LIKE 'Motorcycle%' \
                   GROUP BY Area;"

## Visualizing Fatal Motorcycle Accidents by Area

The total fatal motorcycle accidents of the two areas are being compared, so a simple bar chart will accurately convey the information to the viewer.

In [ ]:
#display visualization in jupyter output inline
%matplotlib inline

#import packages
import matplotlib.pyplot as plt
import pandas as pd

#read csv into dataframe
area = pd.read_csv('~/Desktop/Assignment_files/Reports/QuestionSeven.csv')

#create bar chart
plt.bar(area.area, height=area.fatal_motorcycle_accidents, color='black')
plt.xlabel('Area')
plt.ylabel('# of Fatal Motorcycle Accidents')
plt.title('Fatal Motorcycle Accidents by Area', fontdict = {'fontsize' : 16})
plt.style.use('ggplot')
plt.show()

## Conclusions

The above chart clearly shows that the most fatal motorcycle accidents occur in rural areas. 

# Additional Advice

In addition to the questions posed by the client, the consultant has included advice that should help the client extract even more useful insight from the data provided. Additional visualization techniques are discussed below.

# Casualty Percentage by Casualty Class

There are three casualty severity types in the casualties table: Fatal, Serious, and Slight. It would be interesting to see the percentage breakdown of these severity types by casualty class, which includes 'Driver or Rider', 'Passenger' or 'Pedestrian'. By visualizing this information, it can be seen which casualty class has the highest percentage of fatal or serious accidents. 

## Query to Find Casualty Severity by Casualty Class

The following query finds casualty class by severity. All variables of interest are located in the 'casualties' table.

In [ ]:
#query to find casualty severity by casualty class
! impala-shell -q "SELECT IF(c1.casualty_class = 1, 'Driver or Rider', \
                            IF(c1.casualty_class = 2, 'Passenger', 'Pedestrian')) AS Casualty_Class, \
                          IF(c1.casualty_severity = 1, 'Fatal', \
                            IF(c1.casualty_severity = 2, 'Serious', 'Slight')) AS Accident_Severity, \
                          COUNT(*) AS Casualty_Count \
                   FROM roadsafety.casualties AS c1 \
                   GROUP BY Casualty_Class, Accident_Severity \
                   ORDER BY Casualty_Class, Accident_Severity;"

#export to CSV
! impala-shell -B \
               -o ~/Desktop/Assignment_files/Reports/CasualtyByType.csv \
               --print_header \
               --output_delimiter=',' \
               -q "SELECT IF(c1.casualty_class = 1, 'Driver or Rider', \
                            IF(c1.casualty_class = 2, 'Passenger', 'Pedestrian')) AS Casualty_Class, \
                          IF(c1.casualty_severity = 1, 'Fatal', \
                            IF(c1.casualty_severity = 2, 'Serious', 'Slight')) AS Accident_Severity, \
                          COUNT(*) AS Casualty_Count \
                   FROM roadsafety.casualties AS c1 \
                   GROUP BY Casualty_Class, Accident_Severity \
                   ORDER BY Casualty_Class, Accident_Severity;"

## Visualizing Casualty Severity by Casualty Class

Using the saved query, python can be utilized to find the percentage of each casualty severity within the casualty class. A subplot can then be created to compare each of the three severity levels within the casualty classes. In addition to using matplotlib [4] and pandas [2], numpy [5] is used to create an array to tie the category labels to.  

In [ ]:
#display visualization in jupyter output inline
%matplotlib inline

#import packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#from matplotlib.ticker import MaxNLocator VET THESE
#from collections import namedtuple

#read csv into dataframe
casType = pd.read_csv('~/Desktop/Assignment_files/Reports/CasualtyByType.csv')

#create percentage by casualty class
casType.loc[casType["casualty_class"] == "Driver or Rider", "casualty_percentage"] = \
    (casType.loc[casType["casualty_class"] == "Driver or Rider", "casualty_count"] /
    sum(casType.loc[casType["casualty_class"] == "Driver or Rider", "casualty_count"]) * 100)

casType.loc[casType["casualty_class"] == "Passenger", "casualty_percentage"] = \
    (casType.loc[casType["casualty_class"] == "Passenger", "casualty_count"] /
    sum(casType.loc[casType["casualty_class"] == "Passenger", "casualty_count"]) * 100)

casType.loc[casType["casualty_class"] == "Pedestrian", "casualty_percentage"] = \
    (casType.loc[casType["casualty_class"] == "Pedestrian", "casualty_count"] /
    sum(casType.loc[casType["casualty_class"] == "Pedestrian", "casualty_count"]) * 100)

#round results
casType.casualty_percentage = casType.casualty_percentage.round(2)

#create separate dataframes for graphing
fatal = casType.loc[casType["accident_severity"] == "Fatal"]
serious = casType.loc[casType["accident_severity"] == "Serious"]
slight = casType.loc[casType["accident_severity"] == "Slight"]

#Chart Results

#increase width of chart 
plt.rcParams['figure.figsize'] = [9, 6]

#set number of casualty class categories
categories = 3

#create the subplot figure
fig, ax = plt.subplots()

#index to map on x axis
index = np.arange(categories)
bar_width = 0.25
opacity = .9

#create each bar chart
fatal_bar = ax.bar(index - bar_width/1.5, fatal.casualty_percentage, bar_width,                
                alpha=opacity, color='Red',
                label='Fatal')

serious_bar = ax.bar(index + bar_width/3, serious.casualty_percentage, bar_width,
                alpha=opacity, color='Orange',
                label='Serious')

slight_bar = ax.bar(index + bar_width + bar_width/3, slight.casualty_percentage, bar_width,
                alpha=opacity, color='Yellow',
                label='Slight')

#Set label information
ax.set_xlabel('Casualty Class', fontdict = {'fontsize' : 16})
ax.set_ylabel('Casualty Severity Percentage', fontdict = {'fontsize' : 16})
ax.set_title('Casualty Severity by Class', fontdict = {'fontsize' : 24})
ax.set_xticks(index + bar_width / 3)
ax.set_xticklabels(('Driver or Rider', 'Passenger', 'Pedestrian'), fontdict = {'fontsize' : 12} )
ax.legend()
plt.style.use('ggplot')

#show results
fig.tight_layout()
plt.show()

## Conclusions

As seen in the plot above, pedestrians involved in accidents have the highest percentage of fatal and serious accidents. Since they don't have the protection of a vehicle, injuries will typically be much worse. It is also seen that drivers involved in accidents have a higher percentage of fatal and serious accidents compared to passengers. 

# Age of Driver in Fatal Accidents on Wet Roads

A swarmplot can be used to visualize the age of the driver in all fatal accidents that occured on wet roads. This plot could show if drivers become more cautious in bad conditions as they get older.

## Query to Find Age of Driver in Fatal Accidents on Wet Roads

By joining the 'vehicles' and 'accidents' tables, the age of the driver involved in fatal accidents on wet roads can be found. A road surface condition of '2' indicates a wet or damp road, while an accident severity of 1 indicates that it was a fatal accident.

In [ ]:
#query to find age of drive in fatal accidents on wet roads
! impala-shell -q "SELECT a.Road_Surface_Conditions, \
                          v.Age_of_Driver \
                   FROM roadsafety.accidents AS a \
                     INNER JOIN roadsafety.vehicles AS v \
                     ON a.accident_index = v.accident_index \
                   WHERE v.Age_of_Driver <> -1 \
                         AND a.Road_Surface_Conditions = 2 \
                         AND a.accident_severity = 1 \
                   LIMIT 5;"

#export to CSV
! impala-shell -B \
               -o ~/Desktop/Assignment_files/Reports/WetRoad.csv \
               --print_header \
               --output_delimiter=',' \
               -q "SELECT a.Road_Surface_Conditions, \
                          v.Age_of_Driver \
                   FROM roadsafety.accidents AS a \
                     INNER JOIN roadsafety.vehicles AS v \
                     ON a.accident_index = v.accident_index \
                   WHERE v.Age_of_Driver <> -1 \
                         AND a.Road_Surface_Conditions = 2 \
                         AND a.accident_severity = 1;"

## Visualizing Age of Driver in Fatal Accidents on Wet Roads

Using the package seaborn [6], a swarmplot will nicely display the distribution of age involved in these types of accidents. This type of plot should show the user which age brackets have a tendancy to be involved in accidents involving wet or slippery roads. 

In [ ]:
#display visualization in jupyter output inline
%matplotlib inline

#import packages
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set()

#read csv into dataframe
wetRoad = pd.read_csv('~/Desktop/Assignment_files/Reports/WetRoad.csv')

#increase width of chart 
plt.rcParams['figure.figsize'] = [15, 8]
sns.swarmplot(x=wetRoad["age_of_driver"])
plt.xlabel("Driver's Age", fontdict = {'fontsize' : 20})
plt.title('Fatal Accidents on Wet Roads', fontdict = {'fontsize' : 34})
plt.show()

## Conclusions

As seen above, drivers in their 20's and 30's have the highest frequency of fatal accidents on wet roads. This may imply that with age comes wisdom, at least when driving in bad conditions. 

# Mapping Accident Severity by Longitude and Latitude

The data includes the location of the accident represented by longitude and latitude. A query can be created that maps accident severity by location. By assigning a different color to each of the accident severity categories, a map of the United Kingdom can show which areas have the most severe accidents. 

## Query to Map Accident Severity by Location

The query below isolates three variables: longitude, latitude, and accident severity. Only data points with a valid location are included in the results. 

In [ ]:
#query to find Longitude and Latitude by accident severity
! impala-shell -q "SELECT IF(accident_severity = 1, 'Fatal', \
                          IF(accident_severity = 2, 'Serious', 'Slight')) AS Accident_Severity, \
                          Longitude, \
                          Latitude \
                   FROM roadsafety.accidents \
                   WHERE longitude IS NOT NULL AND latitude IS NOT NULL \
                   LIMIT 10;"

#export to CSV
! impala-shell -B \
               -o ~/Desktop/Assignment_files/Reports/GeoMap.csv \
               --print_header \
               --output_delimiter=',' \
               -q "SELECT IF(accident_severity = 1, 'Fatal', \
                          IF(accident_severity = 2, 'Serious', 'Slight')) AS Accident_Severity, \
                          Longitude, \
                          Latitude \
                   FROM roadsafety.accidents \
                   WHERE longitude IS NOT NULL AND latitude IS NOT NULL;"

## Visualizing Accident Severity by Location

Now that the query is saved, the package cartopy [7] can be used to map the data. For some reason, "sudo pip install cartopy" does not install the package correctly, and to get it working the consultant needed to install the package using Anaconda. The following code will install Anaconda and then use the "conda" command to correctly install the package. This must be done in the terminal as Jupyter Notebook cannot interact with the program as it downloads.

## Install Anaconda to install cartopy package

In [ ]:
# First download the 2.7 anaconda package from https://www.anaconda.com/distribution/#linux [8]
# and save in the Documents folder
!wget -P ~/Documents/ https://repo.anaconda.com/archive/Anaconda2-2019.03-Linux-x86_64.sh

# the following runs the install, follow the prompts in the terminal
# ! bash ~/Documents/Anaconda2-2019.03-Linux-x86_64.sh

# after anaconda is successfully installed, the following installs the cartophy package
# (need to close and reopen the terminal after conda install)
# conda install -c conda-forge cartopy

## Visualizing Accident Severity by Location with Cartopy

The OSGB projection is used to create an outline of Great Britain, and the coordinate information is layed on top of this projection. Instructions on how to use this package were found at the cartopy documentation. [9] 

<span style="color:red">(Need to run this code twice for the map to appear for some reason)</span>

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

#read csv into dataframe
geoMap = pd.read_csv('~/Desktop/Assignment_files/Reports/GeoMap.csv')

#create dataframe for each severity
fatal = geoMap[geoMap["accident_severity"]=="Fatal"]
serious = geoMap[geoMap["accident_severity"]=="Serious"]
slight = geoMap[geoMap["accident_severity"]=="Slight"]

#map data
plt.figure(figsize=(8, 11))
ax = plt.axes(projection=ccrs.OSGB())
ax.coastlines(resolution='50m')
ax.plot(slight.longitude, slight.latitude, 'o', color='yellow', markersize=1, alpha= .6, 
        label="Yellow = Slight", transform=ccrs.Geodetic())
ax.plot(serious.longitude, serious.latitude, 'o', color='orange', markersize=1, alpha= .6, 
        label="Orange = Serious", transform=ccrs.Geodetic())
ax.plot(fatal.longitude, fatal.latitude, 'o', color='red', markersize=1, alpha= .6, 
        label="Red = Fatal", transform=ccrs.Geodetic())
plt.legend(fancybox=True, edgecolor="black", prop={'size': 12})
plt.title('Accidents by Severity in the UK', fontdict = {'fontsize' : 18})
plt.show()

## Conclusions

As seen above, most accidents happen in the south of England. Looking at the map, it is apparent that a high concentration of serious and fatal accidents happen around the London area. Given the high population that lives in those areas, this is to be expected. More people typically means more accidents, and many of the concentrations of accidents above translate to densely populated areas.

# Final Conclusion

The above brief addressed all 7 of the questions posed by the client, and even offered 3 additional visualizations the client might find useful. Sqoop was used to move the data into the HDFS and Impala leveraged big data processing when querying the data. Python visualized the final results, giving the client a full ETL package. The consultant, Ian Jeffries, looks forward to working with the client in the future. 

# References

[1] (n.d.). Retrieved April 29, 2019, from https://data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data4

[2] McKinney, W. (n.d.). Powerful Python data analysis toolkit¶. Retrieved May 2, 2019, from https://pandas.pydata.org/pandas-docs/stable/

[3] Datetime - Basic date and time types¶. (n.d.). Retrieved May 2, 2019, from https://docs.python.org/3/library/datetime.html

[4] Hunter, J. (n.d.). Overview¶. Retrieved May 2, 2019, from https://matplotlib.org/contents.html

[5] Hugunin, J. (n.d.). NumPy¶. Retrieved May 2, 2019, from https://www.numpy.org/

[6] Statistical data visualization¶. (n.d.). Retrieved May 2, 2019, from https://seaborn.pydata.org/

[7] Introduction¶. (n.d.). Retrieved May 2, 2019, from https://scitools.org.uk/cartopy/docs/v0.16/

[8] Anaconda Python/R Distribution. (n.d.). Retrieved May 2, 2019, from https://www.anaconda.com/distribution/#linux

[9] Features¶. (n.d.). Retrieved April 29, 2019, from https://scitools.org.uk/cartopy/docs/latest/gallery/features.html#sphx-glr-gallery-features-py